In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from operator import attrgetter
%matplotlib inline

In [2]:
# Загрузка исходных данных
customers = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/FUi5uSd6BfG_ig')
orders    = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/t9Li4JOfgxuUrg')
items     = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw')

### Описание данных таблицы customers
- customer_id — позаказный идентификатор пользователя (аналог номера паспорта)
- customer_unique_id —  уникальный идентификатор пользователя
- customer_zip_code_prefix —  почтовый индекс пользователя
- customer_city —  город доставки пользователя
- customer_state —  штат доставки пользователя

In [3]:
# Описание струкутры данных таблицы с с уникальными идентификаторами пользователей
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


Стоит отметить, что никаких явных проблем с данными нет, пропущенные значения отсутствуют, типы данных не требуют изменений

### Описание данных таблицы orders
- order_id —  уникальный идентификатор заказа (номер чека)
- customer_id —  позаказный идентификатор пользователя (аналог номера паспорта)
- order_status —  статус заказа
- order_purchase_timestamp —  время создания заказа
- order_approved_at —  время подтверждения оплаты заказа
- order_delivered_carrier_date —  время передачи заказа в логистическую службу
- order_delivered_customer_date —  время доставки заказа
- order_estimated_delivery_date —  обещанная дата доставки

In [4]:
# Описание струкутры данных таблицы заказов
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


- В данной таблице заметны пропущенные значения в колонках с датами, которые могут быть связаны с особенностями работы магазина и формирования цепочки исполнения заказа.
- Явно заметна проблема с типами данных, большинство колонок хранят в себе данные о времени, но представлены в строковом типе данных.

In [5]:
# Перевод необходимых колонок к временному типу данных
orders['order_purchase_timestamp']      = pd.to_datetime(orders.order_purchase_timestamp)
orders['order_approved_at']             = pd.to_datetime(orders.order_approved_at)
orders['order_delivered_carrier_date']  = pd.to_datetime(orders.order_delivered_carrier_date)
orders['order_delivered_customer_date'] = pd.to_datetime(orders.order_delivered_customer_date)
orders['order_estimated_delivery_date'] = pd.to_datetime(orders.order_estimated_delivery_date)

In [6]:
# Самопроверка
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

### Описание данных таблицы items
- order_id —  уникальный идентификатор заказа (номер чека)
- order_item_id —  идентификатор товара внутри одного заказа
- product_id —  ид товара (аналог штрихкода)
- seller_id — ид производителя товара
- shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
- price —  цена за единицу товара
- freight_value —  вес товара

In [7]:
# Описание струкутры данных таблицы товарных позиций, входящих в заказы
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


Стоит отметить, что никаких явных проблем с данными нет, пропущенные значения отсутствуют, только одна колонка требует изменение типа данных - shipping_limit_date, так как она отражает собой дату.

In [8]:
# Перевод необходимой колонки к временному типу данных
items['shipping_limit_date'] = pd.to_datetime(items.shipping_limit_date)

In [9]:
# Самопроверка
items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

# Задание 1
Сколько у нас пользователей, которые совершили покупку только один раз?

Для анализа покупок пользователей нам необходимо связать таблицы **customers** и **orders**, чтобы иметь доступ к **уникального id клиента**. Для реализации данной цели воспользуемся методом **merge**.

In [10]:
# left join датафреймов с помощью метода merge
customers_orders_df = orders.merge(customers, how='left', on='customer_id')

### Анализ проблемы
Для решения поставленной задачи сперва необходимо понять, что можно считать за "покупку". Интерпретировать данную категорию в нашем случае можно несколькими способами:
1. Мы считаем за покупку все заказы, которые имеют любой статус (**order_status**), кроме **"unavailable"** и **"canceled"**. 
    - **Unavailable** можно интерпретировать как "пропавший заказ" или заказ, с которым произошла ошибка в цепочке исполнения заказа. 
    - **Canceled** говорит нам о том, что заказ был отменен. Как правило, заказ возможно отменить на любой стадии его исполнения, также, если отменить его после оплаты, то магазин вернет деньги клиенту, что позволяет нам не считать отмененные заказы за покупку.


2. Мы считаем за покупку все заказы, которые имеют любой статус (**order_status**), кроме **"created"**, **"unavailable"** и **"canceled"**. Логика исключения **created** состоит в том, что заказы с таким статусом не оплачены, соответственно нет смысла считать их за покупку.


3. Мы считаем за покупку все заказы, которые имеют время подтверждения оплаты заказа (**order_approved_at**). Логика проста: если нам поступили деньги за заказ, то покупка совершена.

Проверим целесообразность вышеописанных методов, проведя исследовательский анализ данных с точки зрения статусов заказа (**order_status**).

In [11]:
# Общее количество пропущенных значений
customers_orders_df.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
dtype: int64

In [12]:
# Расмотрим заказы со статусом unavailable
customers_orders_df.query("order_status == 'unavailable'").isna().sum()

order_id                           0
customer_id                        0
order_status                       0
order_purchase_timestamp           0
order_approved_at                  0
order_delivered_carrier_date     609
order_delivered_customer_date    609
order_estimated_delivery_date      0
customer_unique_id                 0
customer_zip_code_prefix           0
customer_city                      0
customer_state                     0
dtype: int64

**Выводы**: все заказы оплачены, но ни один не поступил в логистическую службу, следовательно данные заказы мы можем считать за покупку.

In [13]:
# Расмотрим заказы со статусом canceled
customers_orders_df.query("order_status == 'canceled'").isna().sum()

order_id                           0
customer_id                        0
order_status                       0
order_purchase_timestamp           0
order_approved_at                141
order_delivered_carrier_date     550
order_delivered_customer_date    619
order_estimated_delivery_date      0
customer_unique_id                 0
customer_zip_code_prefix           0
customer_city                      0
customer_state                     0
dtype: int64

**Выводы**: мысль о том, что заказ возможно отменить на любой стадии его исполнения подтвердилась. Из информации о данных видно, что присутствуют заказы, которые были отменены до оплаты, после оплаты и даже после того, как покупатель получил заказ (то есть клиент оформил возврат). Так как отмена или возврат заказа на любом этапе предполагает возврат денег клиенту (если заказ был оплачен), то заказы со статусом **canceled** мы можем не считать за покупку.

In [14]:
# Расмотрим заказы со статусом created
customers_orders_df[['order_status', 'order_approved_at']].query("order_status == 'created'").isna().sum()

order_status         0
order_approved_at    5
dtype: int64

**Вывод**: все заказы со статусом **created** не оплачены, что позволяет нам не считать из за покупку.

In [15]:
# Расмотрим заказы со статусом delivered
customers_orders_df.query("order_status == 'delivered'").isna().sum()

order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
customer_unique_id                0
customer_zip_code_prefix          0
customer_city                     0
customer_state                    0
dtype: int64

**Вывод**: так как 14 заказов не имеют время подтверждения оплаты заказа (**order_approved_at**), 2 не имеют времени передачи заказа в логистическую службу (**order_delivered_carrier_date**), 8 не имеют подтвержденного времени доставки заказа (**order_delivered_customer_date**), разумно предположить, что статус **delivered** некоторым заказам был присвоен ошибочно.

### Локальное заключение
Проведя исследовательский анализ данных можно утверждать, что за "покупку" разумно считать заказы с любым статусом заказа (**order_status**), кроме **created** и **canceled**, а так же заказы, которые имеют время подтверждения оплаты заказа (**order_approved_at**).

### Замечание:
Так как пропущенные значения имеют колонки с датами, то заполним пропуски нулями, для упрощения фильтрации данных.

In [16]:
# Заполнение пропусков
customers_orders_df = customers_orders_df.fillna(0)

In [17]:
# Посчитаем количество покупок, которое совершил каждый пользователь
purchase_df = customers_orders_df \
    .query("order_status != 'created' & \
            order_status != 'canceled' & \
            order_approved_at != 0") \
    .groupby('customer_unique_id', as_index = False) \
    .agg({'order_id': 'count'}) \
    .rename(columns={'order_id': 'purchases'})

In [18]:
purchase_df.purchases.value_counts()

1     92621
2      2686
3       187
4        30
5         8
6         6
7         3
17        1
9         1
Name: purchases, dtype: int64

### Итог:
У нас **92621** пользователей, которые совершили покупку только один раз.